In [ ]:
from importlib import reload
import ipynb.fs.full.viewer
reload(ipynb.fs.full.viewer)
from ipynb.fs.full.viewer import Viewer, root, bucket
import pickle
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
import subprocess
from subprocess import PIPE
from tqdm import tqdm
from time import time
old = set(Path(root/"danbooru2019/512px.dat").read_text().strip().split())
safe =  set((root/"danbooru2020/512px.dat").read_text().strip().split())
i = 0
if not "ids" in globals():
    with timer(prefix="read_id_pkl"): # 311[s]
        with open((root/f"{i}_id.pkl"), "rb") as f:
            ids = pickle.load(f)
if not "tags" in globals():
    with timer(prefix="read_tag_pkl"): # 58[s]
        with open((root/f"{i}_tag.pkl"), "rb") as f:
            tags = pickle.load(f)

In [ ]:
fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())

In [ ]:
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    print("{:,}\t {}".format(len(retval), s))
    return retval
def id2fname(_id, prefix="512white"):
    return str(root/f"danbooru2020/{prefix}/{bucket(_id)}/{_id}.png")

In [ ]:
solo = t("1girl|solo|1boy")
standing = t("standing&full_body&highres")
white = t("white_background|transparent_background")
exc = t('|'.join([
"2boys|2girls|3boys|3girls|4boys|4girls|5boys|5girls|6+boys|6+girls|multiple_boys|multiple_girls",
"car|bicycle",
"no_humans|arachne|robot|pokemon_(creature)|octoling|cyborg|monster_girl",
"blue_skin|purple_skin|wrinkled_skin|green_skin|two-tone_skin",
"monochrome|lineart|text_only_page|photo-referenced|3d",
"multiple_views|variations|zoom_layer",
"full_armor|weapon|chibi|cannon|mask",
"back|ass|from_behind|from_above|bent_over|dutch_angle",
"translation_request"
]))|reduce(set.__or__, fire_eggs.values())

v3 = (safe&solo&standing&white)-exc
print("{:,}\t {}".format(len(v3), "v3"))

In [ ]:
viewer = Viewer(map(id2fname, v3))
viewer

In [ ]:
for k in ids["4254102"]["tags"]:
    print(k["name"])

In [ ]:
horns = Viewer(map(id2fname, tags["horns"]&v3))
horns